# SF-DAT-21 | Unit Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in Unit Projects 1 and 2.

In [94]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pylab as pl
%matplotlib inline

In [2]:
df_raw = pd.read_csv("../../dataset/admissions.csv")
df = df_raw.dropna()
print df.head()

   admit  gre   gpa  prestige
0      0  380  3.61         3
1      1  660  3.67         3
2      1  800  4.00         1
3      1  640  3.19         4
4      0  520  2.93         4


## Part 1. Frequency Tables

#### Question 1. Let's create a frequency table of our variables.

In [18]:
# frequency table for prestige and whether or not someone was admitted
df.groupby("prestige").agg([sum])["admit"]


,sum
prestige,
1,33
2,53
3,28
4,12


## Part 2. Return of dummy variables

#### Question 2.1. Create class or dummy variables for prestige.

In [11]:
dummy_ranks = pd.get_dummies(df.prestige,prefix = "prestige")
dummy_ranks.head()

,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0,0,1,0
1,0,0,1,0
2,1,0,0,0
3,0,0,0,1
4,0,0,0,1


#### Question 2.2. When modeling our class variables, how many do we need?

Answer: We need one less dummy than the number of levels within a factor.  So, prestige has 4 levels (1, 2, 3, 4), so we would model it with 3 dummies (corresponding to 2, 3, and 4, for instance.)

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [12]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_1.0':])
print handCalc.head()

   admit  gre   gpa  prestige_1.0  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380  3.61             0             0             1             0
1      1  660  3.67             0             0             1             0
2      1  800  4.00             1             0             0             0
3      1  640  3.19             0             0             0             1
4      0  520  2.93             0             0             0             1


In [20]:
# crosstab prestige 1 admission
# frequency table cutting prestige and whether or not someone was admitted
cross_tab = df.groupby("prestige").agg([sum])["admit"]
cross_tab

,sum
prestige,
1,33
2,53
3,28
4,12


In [48]:
cross_tab=pd.crosstab(df.prestige,df.admit)
cross_tab

admit,0,1
prestige,,
1,28,33
2,95,53
3,93,28
4,55,12


In [66]:
1.0*cross_tab.iloc[1:4][1].sum()/cross_tab.iloc[1:4].sum().sum()

0.2767857142857143

In [64]:
cross_tab.iloc[1:4].sum().sum()

336

#### Question 3.1. Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college.

In [69]:
odds_1 = 1.0*cross_tab.iloc[0][1]/cross_tab.iloc[0].sum()

#### Question 3.2. Now calculate the odds of admission if you did not attend a #1 ranked college.

In [70]:
odds_not1 = 1.0*cross_tab.iloc[1:4][1].sum()/cross_tab.iloc[1:4].sum().sum()

#### Question 3.3. Calculate the odds ratio.

In [71]:
odds_1/odds_not1

1.9545214172395557

#### Question 3.4. Write this finding in a sentenance:

Answer: You are about 95% more likely to be admitted to grad school if you attended a #1 ranked college than if  you didn't.

#### Question 3.5. Print the cross tab for prestige_4.

In [73]:
handCalc.head()

,admit,gre,gpa,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0,380,3.61,0,0,1,0
1,1,660,3.67,0,0,1,0
2,1,800,4.00,1,0,0,0
3,1,640,3.19,0,0,0,1
4,0,520,2.93,0,0,0,1


In [75]:
cross_tab = handCalc.groupby("prestige_4.0").agg([sum])["admit"]
cross_tab

,sum
prestige_4.0,
0,114
1,12


In [80]:
cross_tab.sum()

sum    126
dtype: int64

#### Question 3.6. Calculate the OR.

In [82]:
1.0*cross_tab.iloc[1]/cross_tab.iloc[0]

sum    0.105263
dtype: float64

#### Question 3.7. Write this finding in a sentence.

Answer: You are 90% less likely to be admitted to grad school if you attended a #4 ranked school than if you didn't.

## Part 4. Analysis

In [83]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2':])
print data.head()

   admit  gre   gpa  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380  3.61             0             1             0
1      1  660  3.67             0             1             0
2      1  800  4.00             0             0             0
3      1  640  3.19             0             0             1
4      0  520  2.93             0             0             1


We're going to add a constant term for our Logistic Regression.  The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [85]:
# manually add the intercept
data['intercept'] = 1.0
data.head()

,admit,gre,gpa,prestige_2.0,prestige_3.0,prestige_4.0,intercept
0,0,380,3.61,0,1,0,1
1,1,660,3.67,0,1,0,1
2,1,800,4.00,0,0,0,1
3,1,640,3.19,0,0,1,1
4,0,520,2.93,0,0,1,1


#### Question 4.1. Set the covariates to a variable called train_cols.

In [92]:
train_cols = data[[1,2,3,4,5,6]]

In [103]:
data.columns = ["admit","gre","gpa","prestige_2","prestige_3","prestige_4","intercept"]
data.columns

Index([u'admit', u'gre', u'gpa', u'prestige_2', u'prestige_3', u'prestige_4',
       u'intercept'],
      dtype='object')

#### Question 4.2. Fit the model.

In [107]:
mod1 = smf.ols("admit ~ gre+gpa+prestige_2+prestige_3+prestige_4+intercept+0",data=data).fit()

#### Question 4.3. Print the summary results.

In [108]:
mod1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  admit   R-squared:                       0.099
Model:                            OLS   Adj. R-squared:                  0.087
Method:                 Least Squares   F-statistic:                     8.594
Date:                Thu, 24 Mar 2016   Prob (F-statistic):           9.71e-08
Time:                        17:47:05   Log-Likelihood:                -239.02
No. Observations:                 397   AIC:                             490.0
Df Residuals:                     391   BIC:                             513.9
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
gre            0.0004      0.000      1.997      0.047      6.48e-06     0.001
gpa            0.1508      0.064      2.349      0.019         0.025     0.277
prestige_2    -0.1635      0.068     -2.407      0.017        -0.297    -0.030
prestige_3    -0.2910      0.070     -4.139      0.000        -0.429    -0.153
prestige_4    -0.3240      0.079     -4.082      0.000        -0.480    -0.168
intercept     -0.2377      0.217     -1.095      0.274        -0.665     0.189
==============================================================================
Omnibus:                      152.312   Durbin-Watson:                   1.946
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               50.314
Skew:                           0.678   Prob(JB):                     1.19e-11
Kurtosis:                       1.904   Cond. No.                     6.07e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.07e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

#### Question 4.4. Calculate the odds ratios of the coeffincients and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params

        conf.columns = ['2.5%', '97.5%', 'OR']

In [110]:
OR = np.exp(mod1.params)

gre           1.000422
gpa           1.162792
prestige_2    0.849131
prestige_3    0.747545
prestige_4    0.723254
intercept     0.788400
dtype: float64

In [136]:
np.exp(mod1.conf_int())

,0,1
gre,1.000006,1.000837
gpa,1.024866,1.319280
prestige_2,0.742965,0.970467
prestige_3,0.651050,0.858342
prestige_4,0.618748,0.845411
intercept,0.514525,1.208055


In [135]:
conf = pd.DataFrame()
conf["OR"] = np.exp(mod1.params)
conf["2.5%"] = np.exp(mod1.conf_int())[[0]]
conf["97.5%"] = np.exp(mod1.conf_int())[[1]]


conf

,OR,2.5%,97.5%
gre,1.000422,1.000006,1.000837
gpa,1.162792,1.024866,1.319280
prestige_2,0.849131,0.742965,0.970467
prestige_3,0.747545,0.651050,0.858342
prestige_4,0.723254,0.618748,0.845411
intercept,0.788400,0.514525,1.208055


#### Question 4.5. Interpret the OR of Prestige_2.

Answer: You are 85% as likely to be admitted to grad school if you went to a #2 school as compared to a 

#### Question 4.6. Interpret the OR of GPA.

Answer: For each 1 point increase in GPA, you are 16% more likely to be admitted to grad school.

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values.  This will allow us to see how the predicted probability of admission increases/decreases across different variables.  First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa".  This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [137]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [138]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)

print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])

gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)

print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])

# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


In [156]:
combos.columns = ["gre","gpa","prestige","intercept"]

In [157]:
combos.head()

,gre,gpa,prestige,intercept
0,220,2.260000,1,1
1,220,2.260000,2,1
2,220,2.260000,3,1
3,220,2.260000,4,1
4,220,2.453333,1,1


#### Question 5.1. Recreate the dummy variables.

In [163]:
# recreate the dummy variables
new_dummies = pd.get_dummies(combos.prestige,prefix = "prestige")
new_dummies.columns = ["prestige_1","prestige_2","prestige_3","prestige_4"]

# keep only what we need for making predictions
new_dummies = new_dummies[[1,2,3]]

new_dummies.head()

,prestige_2,prestige_3,prestige_4
0,0,0,0
1,1,0,0
2,0,1,0
3,0,0,1
4,0,0,0


In [184]:
new_frame = pd.concat([combos[["gre","gpa"]],new_dummies,combos[["intercept"]]],axis=1)

#### Question 5.2. Make predictions on the enumerated dataset.

In [186]:
yhat = mod1.predict(new_frame)

#### Question 5.3. Interpret findings for the last 4 observations.

In [188]:
new_frame.tail(4)

,gre,gpa,prestige_2,prestige_3,prestige_4,intercept
396,800,4,0,0,0,1
397,800,4,1,0,0,1
398,800,4,0,1,0,1
399,800,4,0,0,1,1


In [195]:
yhat[396:400]

array([ 0.70280592,  0.53926377,  0.41184482,  0.37881158])

Answer: For the last 4 observations, our model is predicting a 70.2%, 53.9%, 41.2%, and 37.9% chance of admittance to grad school, respectively.

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.